In [1]:
import pickle
import numpy as np

In [14]:
# load in example file
with open('../forward_data/forward_simulation_data_test0.pkl', 'rb') as f:
    data, earth = pickle.load(f)

In [15]:
x_vals = [data[i]['x'] for i in range(len(data))]
y_vals = [data[i]['y'] for i in range(len(data))]
freqs = data[0]['frequencies']
reals = np.stack([data[i]['hz_real'] for i in range(len(data))])
imags = np.stack([data[i]['hz_imag'] for i in range(len(data))])

In [16]:
# combine the real and imaginary parts
measurements = np.hstack([reals, imags])

In [53]:
# normalize the measurements
norm_measure = (measurements - np.mean(measurements, axis=0)) / np.std(measurements, axis=0)

In [49]:
# check to see if there is a uxo at the location
is_uxo = np.array([earth.layers_at_pt((x_vals[i], y_vals[i]))[0].shape[0] // 2 for i in range(len(x_vals))])

In [51]:
# perform logistic regression

class LogisticRegressionBinary:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Initialize parameters
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Gradient Descent
        for _ in range(self.iterations):
            # Compute model predictions
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            # Compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Update weights and bias
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        # Predict probabilities
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        # Convert probabilities to binary labels
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)

# # Example usage:
# if __name__ == "__main__":
#     # Example data
#     X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
#     y = np.array([0, 0, 1, 1, 1])

#     # Train logistic regression model
#     model = LogisticRegressionBinary(learning_rate=0.01, iterations=1000)
#     model.fit(X, y)

#     # Make predictions
#     predictions = model.predict(X)
#     print("Predictions:", predictions)


In [54]:
regression_model = LogisticRegressionBinary(learning_rate=0.01, iterations=1000)
regression_model.fit(norm_measure, is_uxo)
predictions = regression_model.predict(norm_measure)

In [55]:
predictions - is_uxo

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])